### Importing required libraries

In [25]:
import pandas as pd
import numpy as np

### Loading the provided dataset

In [26]:
df = pd.read_excel("Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Inspecting the dataset

In [27]:
#shape tell how many rows and columns are in the dataset
df.shape

(541909, 8)

In [28]:
#this tell all the information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [29]:
#this tells the number missing value in each column
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [30]:
#this tells percentage of missing values in each column
df.isnull().sum()/len(df) *100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

In [31]:
#this tell what are the distinct countries present in the dataset
df['Country'].nunique()

38

### Data Cleaning

In [32]:
#we have to do customer segmentation so the rows without customerID is useless so this line drops all the rows with missing customerId
df.dropna(subset=['CustomerID'], inplace=True)

In [33]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

the above output show we have managed all the missing values

In [34]:
#this will remove all the orders where the quantity is less than 0, becuase the quantity can't be less than 0
df = df[df['Quantity'] > 0]
df = df[df['UnitPrice'] > 0]

In [35]:
#this will change the datatype of customerId to int
df['CustomerID'] = df['CustomerID'].astype(int)

In [36]:
#this will help in resetting the index of the dataset
df.reset_index(drop=True, inplace=True)

In [37]:
df.shape

(397884, 8)

now the dataset is cleaned and ready for feature engineering

### Featuring Engineering

Now we'll calculate Recency, Frequency, Monetary

Recency - This tells us how recently a customer has purchased(days since last purchase has done by the customer).

Frequency - How often the customer has purchased.

Monetary - This tell how much money the customer has spent.

In [38]:
import datetime as dt
reference_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'Quantity': lambda x: (x * df.loc[x.index, 'UnitPrice']).sum()
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']

rfm.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,326,1,77183.60
12347,2,7,4310.00
12348,75,4,1797.24
12349,19,1,1757.55
12350,310,1,334.40


###  Normalizing rmf

Why normalizing rmf is necessary?

Because right now the recency, frequency and monetary are on very different scales, monetary is very large as compared to others in the table so, if we don't normalize or standardize rmf, K means use euclidean distance, so monetary will dominate.

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

rfm_scaled = scaler.fit_transform(rfm)

rfm_scaled = pd.DataFrame(rfm_scaled, index=rfm.index, columns=rfm.columns)

rfm_scaled.head()

,Recency,Frequency,Monetary
CustomerID,,,
12346,2.334574,-0.425097,8.358668
12347,-0.905340,0.354417,0.250966
12348,-0.175360,-0.035340,-0.028596
12349,-0.735345,-0.425097,-0.033012
12350,2.174578,-0.425097,-0.191347
